In [1]:
import IPython.display as display
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/torch-1.12.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/torchvision-0.13.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmcv-2.0.1-cp310-cp310-manylinux1_x86_64.whl 
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/openmim-0.3.9-py2.py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmengine-0.7.4-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/addict-2.4.0-py3-none-any.whl
!pip install yapf==0.40.1
display.clear_output()

In [2]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

!mkdir checkpoint
!mkdir outputs
!mkdir data
!mkdir data/images
!mkdir data/labels

display.clear_output()

In [3]:
import torch, torchvision,mmdet
print("torch=",torch.__version__,torch.cuda.is_available())
print("mmdet=",mmdet.__version__)
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

torch= 1.12.1+cu116 True
mmdet= 3.3.0
11.6
GCC 9.3


In [4]:
import os
import io
import shutil
from PIL import Image

In [ ]:
!ls /kaggle/input/notebooks/kagglertw/dal-shemagh-2-f4

__notebook__.ipynb  __results__.html   custom.css
__output__.json     __results___files  shemagh


In [ ]:
from pycocotools.coco import COCO


annotation_file = '/kaggle/input/notebooks/kagglertw/dal-shemagh-2-f4/shemagh/train_dataset.json'


coco = COCO(annotation_file)


categories = coco.loadCats(coco.getCatIds())
category_id_to_name = {cat['id']: cat['name'] for cat in categories}


for category_id, category_name in category_id_to_name.items():
    print(f"Category ID: {category_id}, Category Name: {category_name}")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Category ID: 0, Category Name: head
Category ID: 1, Category Name: shemagh


In [7]:
!wget https://download.openmmlab.com/mmdetection/v3.0/ddq/ddq_detr_swinl_30e.pth -P ./checkpoints
display.clear_output()

In [ ]:
from mmengine import Config
cfg = Config.fromfile('configs/ddq/ddq-detr-4scale_swinl_8xb2-30e_coco.py')


In [ ]:
!mkdir -p /kaggle/working/data
!cp -r /kaggle/input/notebooks/kagglertw/data-management-5f/data_5fold/fold_4/* /kaggle/working/data/

In [ ]:
!ls /kaggle/input/notebooks/kagglertw/dal-shemagh-2-f4/shemagh

train  train_dataset.json  val	val_dataset.json


In [12]:
!mkdir /kaggle/working/data

mkdir: cannot create directory '/kaggle/working/data': File exists


In [ ]:
!ls /kaggle/input/notebooks/kagglertw/data-management-5f/data_5fold/fold_4/

train  val


In [17]:
def count_png_files(folder_path):
    count = 0
    for file in os.listdir(folder_path):
        if file.lower().endswith(".jpg"):
            count += 1
    return count

In [18]:
!mkdir /kaggle/working/data/val/images
!mkdir /kaggle/working/data/train/images

In [19]:
!mv /kaggle/working/data/val/*.jpg /kaggle/working/data/val/images/
!mv /kaggle/working/data/train/*.jpg /kaggle/working/data/train/images/

In [20]:
!ls /kaggle/working/data/val/

_annotations.coco.json	images


In [21]:
!ls /kaggle/working/data/train/

_annotations.coco.json	images


In [22]:
!ls /kaggle/working/data

train  val


In [23]:
count_png_files("/kaggle/working/data/val/images")

126

In [24]:
count_png_files("/kaggle/working/data/train/images")

525

In [25]:
config_car = """
_base_ = './ddq-detr-4scale_swinl_8xb2-30e_coco.py'

data_root = '/kaggle/working/data/'

train_batch_size_per_gpu = 1
train_num_workers = 2

max_epochs = 50

metainfo = {
    'classes': ('head', 'shemagh'),
    'palette': [(127,201,127),(190,174,212)]
}


train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=_base_.backend_args),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RandomFlip', prob=0.5),
    dict(type='ColorJitter', brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    dict(type='CLAHE', clip_limit=2.0, tile_grid_size=(8, 8)),
    dict(type='GaussianBlur', sigma_range=(0.1, 2.0)),
    dict(type='RandomAffine', scaling=(0.9, 1.1), translation=(0.1, 0.1), rotate=(-20, 20)),
    dict(type='RandomBrightnessContrast', brightness_limit=0.3, contrast_limit=0.3, prob=0.3),
    dict(type='Sharpen', alpha=(0.1, 0.3), lightness=(0.75, 1.5)),
    
    dict(
        type='RandomChoice',
        transforms=[
            [
                dict(
                    type='RandomChoiceResize',
                    scales=[(1280, 1280)],
                    keep_ratio=True)
            ],
            [
                dict(
                    type='RandomChoiceResize',
                    scales=[(1280, 4200)],
                    keep_ratio=True),
                dict(
                    type='RandomCrop',
                    crop_type='absolute_range',
                    crop_size=(384, 640),
                    allow_negative_crop=True),
                dict(
                    type='RandomChoiceResize',
                    scales=[(1280, 1280)],
                    keep_ratio=True)
            ]
        ]),
    dict(type='PackDetInputs')
]

test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(1280, 1280), keep_ratio=True),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='train/images'),
        ann_file='/kaggle/working/data/train/_annotations.coco.json'))
        
val_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='val/images'),
        ann_file='/kaggle/working/data/val/_annotations.coco.json'))
        
test_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='val/images'),
        ann_file='/kaggle/working/data/val/_annotations.coco.json'))

val_evaluator = dict(ann_file="/kaggle/working/data/val/_annotations.coco.json")

test_evaluator = dict(ann_file="/kaggle/working/data/val/_annotations.coco.json")

model = dict(backbone=dict(frozen_stages=2), bbox_head=dict(num_classes=2))

optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(
        type='AdamW',
        lr=0.0003/16,
        weight_decay=0.05),
    clip_grad=dict(max_norm=0.1/16, norm_type=2),
    paramwise_cfg=dict(custom_keys={'backbone': dict(lr_mult=0.1/16)})
)

default_hooks = dict(
    checkpoint=dict(
        interval=1,
        max_keep_ckpts=2,
        save_best='auto' , 
    ),
    logger=dict(type='LoggerHook', interval=2000))

load_from = './checkpoints/ddq_detr_swinl_30e.pth'

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=2)
randomness = dict(seed=2025, deterministic=False, diff_rank_seed=False)
"""

with open('./configs/ddq/ddq-detr-4scale_swinl_1xb1-15e_sky.py', 'w') as f:
    f.write(config_car)


In [26]:
!python tools/train.py configs/ddq/ddq-detr-4scale_swinl_1xb1-15e_sky.py 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
02/16 19:17:03 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
    CUDA available: True
    numpy_random_seed: 2025
    GPU 0: Tesla P100-PCIE-16GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f